# **MODELO 1 - MODELO DE SIMULACIÓN DE IMÁGENES ANTE LA PRESENCIA DE EVENTOS EN UN SISTEMA COTDR**

In [ ]:
# IMPORTAMOS LAS LIBRERÍAS NECESARIAS:
# Una librería de cálculo numérico y una librería para la representación gráfica
import numpy as np  # Paquete que contiene objetos relacionados con arrays, matrices, y operaciones matemáticas sobre ellos
import scipy
from scipy import signal
from scipy.signal import butter, lfilter  # Métodos que diseñan y filtran señales, respectivamente
from numpy import mean, std  # Métodos que devuelven media y desviación estándar de los objetos de un eje/array determinado de una matriz
import matplotlib.pyplot as plt  # Método para plotear sets de datos
import matplotlib.ticker as ticker
import math  # Módulo que provee acceso a funciones matemáticas
import random  # Este módulo implementa generadores de números pseudoaleatorios para varias distribuciones
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## **PARTE 1 - DEFINICIÓN DEL SISTEMA COTDR E IMPLEMENTACIÓN DE LOS MÉTODOS NECESARIOS OBTENER IMÁGENES Y SIMULAR EVENTOS**

In [ ]:
# DEFINICION DE LA CLASE FIBRA OPTICA

class fibraOptica:

  velocidadPropagacion = 2e8 # m/s (velocidad de la luz en la fibra)
  atenuacionFibra = 0.15 # db/km -> potencia de la onda en km1 = sqrt(2) * potencia de la onda en km0 , porque 10^0.15=sqrt(2)
  longitudRayleigh = 1 # metro
  velocidadPropagacionPerturbaciones = 1000 # m/s

  # MODELO DE ATENUACION OPTICA EN LA FIBRA
    # Consideramos un coeficiente de atenuación óptica uniforme a lo largo de la fibra.
    # La atenuación óptica se representa a través de una función de atenuación para
    # el tiempo de ida y vuelta hasta un punto determinado de la fibra
  @classmethod
  def atenuacionOptica(self, t):
    absorcion = self.atenuacionFibra * self.velocidadPropagacion * t / 1000 # dB
    return 10.0 ** (-absorcion/20.0)

  # MODELO DE EMISORES RAYLEIGH DISCRETOS DISTRIBUIDOS
    # Se posiciona a lo largo de la fibra un emisor en el centro de cada longitudRayleigh.
    # La posición de cada centro de emisión tiene una incertidumbre asociada a una distribución uniforme dentro de la longitudRayleigh.
    # Consideramos un tamaño uniforme para los centros de dispersión.
    # Dada una distribución de centros de emisión, solo un grupo de ellos permanece activo en un tiempo dado:
    # aquellos sobre los que se encuentra el pulso óptico en dicho tiempo.
    # Para que la simulación sea coherente, es necesario que la longitudRayleigh sea mucho menor
    # que la longitud espacial del pulso que interroga al sistema.
    # Aseguramos una ratio máxima de 1/25 en el procedimiento de generación de los centros de emisión Raileigh.

  # INICIALIZACION DE LOS EMISORES
  def __init__(self, COTDR, longitudFibra):
    self.COTDR = COTDR
    self.longitudFibra = longitudFibra # metros
    self.nEmisoresRayleigh = math.floor(self.longitudFibra / self.longitudRayleigh)  # longitudFibra / 1 -> si la fibra mide 1000 metros, hay 1000 emisores Rayleigh
    self.posEmisoresRayleigh = self.longitudRayleigh*(np.arange(self.nEmisoresRayleigh) + 0.5) # Situamos un emisor en el centro de cada longitud Rayleigh
    self.posEmisoresRayleigh += np.random.default_rng().normal(0, self.longitudRayleigh/4, self.nEmisoresRayleigh) # Variamos la posición de cada emisor según una distribución aleatoria
    self.strainVariationEmisoresRayleigh = np.zeros_like(self.posEmisoresRayleigh) # Inicializamos una variable que contiene las variaciones que se van a producir en el futuro sobre las posiciones de los emisores
    self.tActivacionEmisores = 2/self.velocidadPropagacion * self.posEmisoresRayleigh # Calculamos los tiempos que le cuesta a un pulso óptico llegar a cada emisor
  
  def calculaDistanciaEventoFibra (self, posicionesXFibra, posicionYFibra, posicionZFibra, xEvento, yEvento):  # Duda: sería buena idea colocar la fibra en (x,y,z)=(x,L/2,L/2)?
    xFibra = posicionesXFibra
    yFibra = np.full_like(xFibra, posicionYFibra)  # La fibra se colocará horizontal, es decir, paralela al eje x
    zFibra = np.full_like(xFibra, posicionZFibra)
    posicionesFibra = np.vstack([xFibra, yFibra, zFibra])  # Aquí guardamos posiciones en x y en y de los puntos de la fibra, en dos filas
    posicionEvento = np.vstack([[xEvento] , [yEvento] , [0]])  # Colocamos el plano en el que suceden los eventos en z=0
    distanciasEvento_Fibra = abs(posicionesFibra-posicionEvento)
    distanciasAbsolutasEvento_Fibra = np.sqrt(distanciasEvento_Fibra[0]**2 + distanciasEvento_Fibra[1]**2 + distanciasEvento_Fibra[2]**2)
    return distanciasAbsolutasEvento_Fibra  # Devuelve un array 1D con las distancias desde el evento hasta los puntos discretos de la fibra

  def impulsoEscalon (self, t0Abs, tfAbs, tiempoEntrePasos, tSubida, tPisada, t):  # 1) numeroPasos = (tfAbs - t0Abs)/tiempoEntrePasos , 2) t = np.linspace(0, 60 seg, 60/timeStep)
    def Paso(generalTSub, generalTPis):
        pendienteSubida = 1 / generalTSub
        t1 = generalTSub
        t2 = generalTSub + generalTPis
        t3 = 2*generalTSub + generalTPis
        return  lambda t: (t<0)*0 + ((t>=0)&(t<t1))*t*pendienteSubida + ((t>=t1)&(t<t2))*1 + ((t>=t2)&(t<t3))*(1-(t-t2)*pendienteSubida) + (t>=t3)*0
    paso = Paso(tSubida, tPisada)  # Así generamos un paso genérico, con pulso en forma de escalón
    t0 = np.arange(t0Abs, tfAbs, tiempoEntrePasos)  # t0Abs y tfAbs son los tiempos en los que se producen el primer y el último paso, respectivamente, cada un tiempoEntrePasos
    t0 = t0 + 0.25*np.random.random(t0.shape)  # Variamos ligeramente el tiempo entre pasos, para que no se produzcan a una frecuencia exacta
    pasos = sum(map(lambda tIni: paso(t-tIni), t0))  # Genera la función con pasos repartidos en el tiempo según la lista t0
    return pasos

  # FUNCION DE VARIACION ALEATORIA DE LAS POSICIONES DE LOS EMISORES RAYLEIGH CON INTENSIDAD CONSTANTE
  def añadeRuidoTermico(self, intensidad): # intensidad normalizada a longitudRayleigh/10
    self.posEmisoresRayleigh += np.random.default_rng().normal(0, intensidad*self.longitudRayleigh/10, self.nEmisoresRayleigh) # Aproximamos el ruido térmico como una variación aleatoria de las posiciones de los emisores
    self.tActivacionEmisores = 2/self.velocidadPropagacion * (self.posEmisoresRayleigh + self.strainVariationEmisoresRayleigh) # Recalculamos los tiempos de llegada del pulso a cada emisor

  # ASUMIMOS QUE LA VARIACION DE LA FUNCION DE INTERFERENCIA SE PRODUCE POR VARIACIONES LOCALES DE DISTANCIA ENTRE EMISORES RAYLEIGH (STRAIN, TEMPERATURA, PRESION, ...)
  # FUNCION DE VARIACION DE POSICIONES DE LOS EMISORES EN TORNO A ZONAS DE DILATACION 
  def añadeDilatacionLocal(self, posicion, anchura, intensidad): # Posicion normalizadas a longitudFibra. Intensidad normalizada a longitudRayleigh/10 = 0.1. Anchura en metros
    a0 = intensidad*self.longitudRayleigh/10 # Magnitud máxima de la variación 
    z0 = posicion*self.longitudFibra # Posición de variación máxima
    z = self.posEmisoresRayleigh
    variacionEmisores = a0/(anchura*math.sqrt(2*math.pi)) * np.exp(-1/2 * ((z-z0)/anchura)**2) * np.sign(z-z0) # Aproximamos que la variación de las posiciones de los centros en torno a un máximo sigue una función gaussiana
    self.strainVariationEmisoresRayleigh = variacionEmisores
    self.tActivacionEmisores = 2/self.velocidadPropagacion * (self.posEmisoresRayleigh + self.strainVariationEmisoresRayleigh) # Recalculamos los tiempos de llegada del pulso a cada emisor

  # ASUMIMOS QUE LA FUNCIÓN DE VARIACIÓN DE LA FUNCIÓN INTERFERENCIA SE PRODUCE POR VARIACIONES DE DISTANCIA ENTRE EMISORES RAYLEIGH POR PRESIÓN
  # FUNCIÓN DE VARIACIÓN DE POSICIONES DE LOS EMISORES EN TORNO A ZONAS DE DILATACIÓN, EN PRESENCIA DE UN EVENTO FIJO DE TIPO: PULSO CUADRADO
  def añadeDilatacionesLocales_evFijo(self, xEvento, distancias, tiemposVuelo, p0, coefAbsorcion, pasos):
      z = self.posEmisoresRayleigh
      a0 = self.longitudRayleigh/10*p0*np.exp(-coefAbsorcion*distancias)*1/distancias * pasos
      variacionEmisores = a0  # *np.sign(z-xEvento)
      self.strainVariationEmisoresRayleigh = variacionEmisores
      self.tActivacionEmisores = 2/self.velocidadPropagacion * (self.posEmisoresRayleigh + self.strainVariationEmisoresRayleigh)

  # FUNCION OBTENCION DE LOS EMISORES RAYLEIGH ILUMINADOS QUE EN UN DETERMINADO TIEMPO CONTRIBUYEN CON SU REFLEXION EN EL DETECTOR
  def emisoresActivos(self, t):
    zonaIluminada = (self.tActivacionEmisores > t) & (self.tActivacionEmisores < t + self.COTDR.duracionPulso) # Los límites de la zona iluminada los marca la posición de la cabeza y de la cola del pulso en un tiempo t
    return (self.posEmisoresRayleigh[zonaIluminada] + self.strainVariationEmisoresRayleigh[zonaIluminada], self.tActivacionEmisores[zonaIluminada]) # Devolvemos tanto las posiciones como los tiempos de activación de los emisores iluminados en un tiempo t

In [ ]:
# DEFINICION DE LA CLASE LASER
class laser:

  lOndaCentral = 1550e-9 # metros
  
  def __init__(self, COTDR, anchuraEmision):
    self.COTDR = COTDR
    self.freqCentral = fibraOptica.velocidadPropagacion/self.lOndaCentral # metros
    self.anchuraEmision = anchuraEmision # Hz

  # MODELO DE ANCHURA ESPECTRAL DE LA FUENTE
    # Consideramos una fuente laser de longitud de onda central de emisión lOndaCentral y con una anchura espectral anchuraEmision
    # correspondiente a una distribución gausiana centrada en la frecuencia de emisión.

  # CONSTRUCCION DE LA SECUENCIA FRECUENCIAL DEL PULSO
  # Para simular la anchura de pulso, estamos asumiendo que durante la duracion del pulso, el laser puede estar cambiando aleatoriamente su frecuencia de emisión
  # Como vamos a utilizar una discretización donde en cada longitud Rayeligh hay un emisor, nos basta con calcular la variación de la frecuencia de emisión del laser en unos pocos tiempos
  # correspondientes más o menos a esa discretización. 
  def generaPulso(self):
    stepTPulso = fibraOptica.longitudRayleigh / fibraOptica.velocidadPropagacion
    pulsoTemporal = np.linspace(0, self.COTDR.duracionPulso, math.floor(self.COTDR.duracionPulso/stepTPulso)) # Lista con unos cuantos tiempos dentro de la duración de pulso
    pulsoChirp = (0e11 / 200e-9)*np.linspace(0, self.COTDR.duracionPulso, math.floor(self.COTDR.duracionPulso/stepTPulso))
    pulsoFrec = self.freqCentral + pulsoChirp + np.random.default_rng().normal(0, self.anchuraEmision, pulsoTemporal.size) # Lista con las frecuencias de emisión correspondientes
    return (pulsoTemporal,pulsoFrec)

In [ ]:
# DEFINICION DE LA CLASE DETECTOR
class detector:

  def __init__(self, COTDR, fMuestreo, fCorteDetector):
    self.COTDR = COTDR
    self.fMuestreo = fMuestreo # Hz
    self.fCorteDetector = fCorteDetector # Hz
  
  # Utilizamos un filtro paso bajo para simular el tiempo de integración del detector
  # Otra opción más sencilla sería no filtrar la salida, y simplemente promediar varias trazas seguidas para simular el tiempo de integración
  def butter_lowpass(self, cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a
  
  def butter_lowpass_filter(self, data, cutoff, fs, order=5):
    b, a = self.butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [ ]:
# DEFINICION DE LA CLASE ESTIMULO
# Clase definida como una colección de funciones estímulo para ser utilizadas en el generador de dilatacion de la fibra
class estimulo:

  def __init__(self, COTDR):
    self.COTDR = COTDR  # Ahora habrá que añadir esto en todos los fragmentos de código en los que no esté

  # En esta aproximación asumimos por ejemplo que un golpe en el suelo en el entorno de la fibra produce una onda acústica con amplitud amortiguada
  # La dilatación en la fibra será producida por esa onda acústica
  @classmethod
  def impulso (self, amplitud, frecuencia, amortiguamiento):
    def estimulo(t,t0): # t0 es el momento a partir del que se aplica la función impulso
      return (t >= t0) * amplitud * np.exp(-amortiguamiento*(t-t0)) * np.sin(frecuencia*(t-t0)) 
    return estimulo

In [ ]:
# DEFINICION DE LA CLASE COTDR
class COTDR:

  def __init__(self, longitudFibra, anchuraEmision, fMuestreo, fCorteDetector, duracionPulso = 200e-9):
    self.laFibra = fibraOptica(COTDR = self, longitudFibra = longitudFibra)
    self.elLaser = laser(COTDR = self, anchuraEmision = anchuraEmision)
    self.elDetector = detector(COTDR = self, fMuestreo = fMuestreo, fCorteDetector = fCorteDetector)
    #self.elEstimulo = estimulo(COTDR = self)
    self.duracionPulso = duracionPulso

  # Asumimos que la señal detectada en un tiempo dado t es la suma interferencial de las
  # reflexiones provenientes de los emisores iluminados correspondientes a dicho tiempo
  def ondaSuma(self, t):
    posEmisoresActivos, tEmisoresActivos = self.laFibra.emisoresActivos(t) # Miramos qué emisores están iluminados (si es que hay alguno) en el tiempo t , y cuales son los tiempos de llegada del pulso a los mismos.
    if (tEmisoresActivos.size > 0):
      desfasesTemporales = np.flip(tEmisoresActivos[-1] - tEmisoresActivos) 
    else:
      desfasesTemporales = np.array([])
    # Con la lista de tiempos y frecuencias de emisión dentro del pulso que hemos obtenido del laser, interpolamos para obtener la frecuencia con la que está iluminado cada emisor en el tiempo t
    frecuenciasPulso = np.interp(desfasesTemporales, self.pulsoTemporal, self.pulsoFrec) 
    lOndaPulso = fibraOptica.velocidadPropagacion / frecuenciasPulso # Calculamos las correspondientes longitudes de onda
    faseReflexionEmisores = 4*np.pi/lOndaPulso * posEmisoresActivos # Calculamos los desfase correspondientes a recorrer la fibra ida y vuelta hasta cada emisor
    campoReflexionEmisores = np.sin(faseReflexionEmisores) * fibraOptica.atenuacionOptica(t) # Calculamos con dichos desfases la colección de ondas reflejadas que tenemos en el receptor en el tiempo t
    return np.sum(campoReflexionEmisores) ** 2 # Calculamos la interferencia de dichas ondas, IMP: V_detector(t) \proportional_to |\sum_of E_i,R|**2 -> es decir, la tensión detectada en el detector es proporcional al módulo al cuadrado del campo eléctrico reflejado total
  
  def generaTraza(self):
    self.pulsoTemporal, self.pulsoFrec = self.elLaser.generaPulso() # Cada traza interferencial corresponde a la retrodispersión que produce el viaje de un único pulso a lo largo de la fibra
    z = np.linspace(0, self.laFibra.longitudFibra, math.floor(self.laFibra.longitudFibra/fibraOptica.velocidadPropagacion * self.elDetector.fMuestreo)) # Calculamos el VECTOR DE POSICIONES correspondiente a discretizar la recepción con un tiempo de muestreo
    deteccion = np.vectorize(self.ondaSuma) # Generamos una función vectorial que se pueda aplicar de golpe a un vector de tiempos
    t = 2 * z / fibraOptica.velocidadPropagacion # Calculamos el VECTOR DE TIEMPOS de ida y vuelta al detector, hasta los puntos en los que hemos discretizado la fibra
    adquisicion = deteccion(t) # Calculamos la interferencia en cada uno de esos tiempos
    traza = self.elDetector.butter_lowpass_filter(adquisicion, self.elDetector.fCorteDetector, 0.5*self.elDetector.fMuestreo) # Suavizamos la traza. (Se podría hacer sumando varias trazas sucesivas)
    return (z,traza) # Devolvemos la interferencia detectada y las correspondientes posiciones en la fibra 

  def generaWaterfallVarianza(self, waterfall , intervaloVarianza):
    m = waterfall
    mm = signal.convolve2d(m , np.ones([intervaloVarianza , 1]) , mode = 'same')
    nn = signal.convolve2d(np.ones([len(m) , len(m[0])]) , np.ones([intervaloVarianza , 1]) , mode = 'same')
    mediaCorrida = mm/nn
    difConMedia2 = (m - mediaCorrida)**2
    sumDifConMedia2 = signal.convolve2d(difConMedia2 , np.ones([intervaloVarianza , 1]) , mode = 'same')
    varianzaCorrida = sumDifConMedia2/nn
    return varianzaCorrida

  def generaWaterfallFourier(self, waterfall, intervaloFourier):
    m = waterfall
    def rollingWindow(a, windowSize):
        shape = (a.shape[0] - windowSize + 1, windowSize) + a.shape[1:]
        strides = (a.strides[0],) + a.strides
        return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)
    windows = rollingWindow(m, intervaloFourier)
    fft = scipy.fft.fft(windows, axis = 1)
    fft = np.delete(fft, 0, axis = 1)  # Borramos la componente correspondiente a frecuencia 0
    fftMax = np.ndarray.max(abs(fft), axis = 1)
    return fftMax

  def generaWaterfallAjuste(self, waterfall, intervaloAjuste, coeficienteAjuste):
    m = waterfall
    def rollingWindow(a, windowSize):
        shape = (a.shape[0] - windowSize + 1, windowSize) + a.shape[1:]
        strides = (a.strides[0],) + a.strides
        return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)
    windows = rollingWindow(m, intervaloAjuste)
    windowsStack = np.hstack(windows)
    coefsStack = np.polyfit(np.arange(intervaloAjuste), windowsStack, coeficienteAjuste)
    previsionStack = np.polyval(coefsStack, intervaloAjuste)
    previsiones = previsionStack.reshape([len(m)-(intervaloAjuste-1) , len(m[0])])
    desviaciones = previsiones[:-1] - m[intervaloAjuste:]
    return (desviaciones)**2

## **PARTE 2 - SIMULACIÓN DE EVENTOS PARA OBTENCIÓN DE N IMÁGENES SINTÉTICAS POR LOS TRES MÉTODOS**

In [ ]:
# NÚMERO DE IMÁGENES A GENERAR DE CADA TIPO DE EVENTO, CON CADA MÉTODO -> N

N = 1

### **2A - RUIDO TÉRMICO**

In [ ]:
# 1.1) SOLO RUIDO | VARIANZA

elCOTDR = COTDR(longitudFibra=100, anchuraEmision=0, fMuestreo=1e9, fCorteDetector=1e8)
t = np.linspace(start=0.12, stop=60, num= int(60/0.12))
numeroImagenes = N

for n in np.arange(numeroImagenes):
    intensidadRuidoTermico = 4e-8 + 4e-8 * np.random.random_sample()
    
    z, traza = elCOTDR.generaTraza()
    waterfall = np.array([traza])

    for tt in t:
        elCOTDR.laFibra.añadeRuidoTermico(intensidadRuidoTermico)
        z, traza = elCOTDR.generaTraza()
        waterfall = np.append(waterfall,[traza],axis=0)  # Matriz m, con trazas interferométricas en los distintos pasos de tiempo
"""
    waterfallVarianza = elCOTDR.generaWaterfallVarianza(waterfall = waterfall , intervaloVarianza = 20)
    #print(np.amax(waterfallVarianza), np.amin(waterfallVarianza))
    #plt.imshow( 10 * waterfallVarianza, vmin=0, vmax=255, cmap='plasma')
    imagePath = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Varianza/Ruido/V" + str(n+1) + "_SoloRuido.png"
    plt.imsave(imagePath, 10 * waterfallVarianza, vmin=0, vmax=255, cmap='plasma')
    filename = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Varianza/Ruido/V%d_SoloRuido.npy" %(n+1)
    np.savetxt(filename, (10*waterfallVarianza).astype('float32'))
"""
    # Generación imagen Varianza
    
    fig=plt.figure(figsize=(2.5,2.5))
    plt.imshow(10*waterfallVarianza, vmin=0, vmax=255, cmap='plasma')
    plt.xticks(np.arange(0,501,100) , np.arange(0,101,20))
    plt.yticks(np.arange(0,502,83) , np.arange(0,61,10))
    plt.xlabel("$z_{fibra}$ (m)", fontsize=12)
    plt.ylabel("$t_{k}$ (s)", fontsize=12)
    plt.savefig('vruido.pdf')

In [ ]:
# 1.2) SOLO RUIDO | AJUSTE

elCOTDR = COTDR(longitudFibra=100, anchuraEmision=0, fMuestreo=1e9, fCorteDetector=1e8)
t = np.linspace(start=0.12, stop=60, num= int(60/0.12))
numeroImagenes = N

for n in np.arange(numeroImagenes):
    intensidadRuidoTermico = 4e-8 + 4e-8 * np.random.random_sample()
    
    z, traza = elCOTDR.generaTraza()
    waterfall = np.array([traza])

    for tt in t:
        elCOTDR.laFibra.añadeRuidoTermico(intensidadRuidoTermico)
        z, traza = elCOTDR.generaTraza()
        waterfall = np.append(waterfall,[traza],axis=0)  # Matriz m, con trazas interferométricas en los distintos pasos de tiempo
"""
    waterfallAjuste = elCOTDR.generaWaterfallAjuste(waterfall = waterfall , intervaloAjuste = 10, coeficienteAjuste = 1)
    #print(np.amax(waterfallAjuste), np.amin(waterfallAjuste))
    #plt.imshow(5 * waterfallAjuste, vmin=0, vmax=255, cmap='plasma')
    imagePath = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Ajuste/Ruido/A" + str(n+1) + "_SoloRuido.png"
    plt.imsave(imagePath, 5 * waterfallAjuste, vmin=0, vmax=255, cmap='plasma')
    filename = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Ajuste/Ruido/A%d_SoloRuido.npy" %(n+1)
    np.savetxt(filename, (5*waterfallAjuste).astype('float32'))
"""
    # Generación imagen Ajuste

    fig=plt.figure(figsize=(2.5,2.5))
    plt.imshow(10*waterfallAjuste, vmin=0, vmax=255, cmap='plasma')
    plt.xticks(np.arange(0,501,100) , np.arange(0,101,20))
    plt.yticks(np.arange(0,492,81) , np.arange(0,61,10))
    plt.xlabel("$z_{fibra}$ (m)", fontsize=12)
    plt.ylabel("$t_{k}$ (s)", fontsize=12)
    plt.savefig('aruido.pdf')

In [ ]:
# 1.3) SOLO RUIDO | FOURIER

factorFourier = 1

elCOTDR = COTDR(longitudFibra=100, anchuraEmision=0, fMuestreo=1e9, fCorteDetector=1e8)
t = np.linspace(start=0.12, stop=60, num= int(60/0.12))
numeroImagenes = N

for n in np.arange(numeroImagenes):
    """intensidadRuidoTermico = 4e-8 + 4e-8 * np.random.random_sample()"""

    intensidadRuidoTermico = 4e-8
    
    z, traza = elCOTDR.generaTraza()
    waterfall = np.array([traza])

    for tt in t:
        elCOTDR.laFibra.añadeRuidoTermico(intensidadRuidoTermico)
        z, traza = elCOTDR.generaTraza()
        waterfall = np.append(waterfall,[traza],axis=0)  # Matriz m, con trazas interferométricas en los distintos pasos de tiempo
"""
    waterfallFourier = elCOTDR.generaWaterfallFourier(waterfall = waterfall , intervaloFourier = 75)
    #print(np.amax(waterfallFourier), np.amin(waterfallFourier))
    #plt.imshow( factorFourier*waterfallFourier, vmin=0, vmax=255, cmap='plasma')
    imagePath = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Fourier/Ruido/F" + str(n+1) + "_SoloRuido.png"
    plt.imsave(imagePath, factorFourier*waterfallFourier, vmin=0, vmax=255, cmap='plasma')
    filename = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Fourier/Ruido/F%d_SoloRuido.npy" %(n+1)
    np.savetxt(filename, factorFourier*(waterfallFourier).astype('float32'))
"""
    # Generación imagen Fourier

    intervaloFourier=75
    fig=plt.figure(figsize=(2.5,7))
    plt.imshow(factorFourier*waterfallFourier, vmin=0, vmax=255, cmap='plasma')
    plt.xticks(np.arange(0,501,100) , np.arange(0,101,20))
    plt.yticks(np.arange(0,427,71) , np.arange(0,61,10))
    plt.xlabel("$z_{fibra}$ (m)", fontsize=12)
    plt.ylabel("$t_{k}$ (s)", fontsize=12)
    plt.savefig('fruido.pdf')

### **2B - EVENTO FIJO + RUIDO TÉRMICO**

In [ ]:
# 2.1) RUIDO + EVENTO FIJO | VARIANZA

elCOTDR = COTDR(longitudFibra=100, anchuraEmision=0, fMuestreo=1e9, fCorteDetector=1e8)
t = np.linspace(start=0.12, stop=60, num= int(60/0.12))
numeroImagenes = N

for n in np.arange(numeroImagenes):

    x0Evento = 10 + (elCOTDR.laFibra.longitudFibra-20) * np.random.random_sample()
    y0Evento = (elCOTDR.laFibra.longitudFibra/2 - 3) + 6 * np.random.random_sample()
    t0Evento = 0 + 15 * np.random.random_sample()
    tfEvento = 45 + 15 * np.random.random_sample()
    intensidadRuidoTermico = 4e-8 + 4e-8 * np.random.random_sample()
    print(x0Evento, y0Evento, t0Evento, tfEvento)
"""
    x0Evento = elCOTDR.laFibra.longitudFibra/2
    y0Evento = elCOTDR.laFibra.longitudFibra/2+1
    t0Evento = 0
    tfEvento = 60
    intensidadRuidoTermico = 6e-8
"""
    zEmisores = elCOTDR.laFibra.posEmisoresRayleigh
    distancias = elCOTDR.laFibra.calculaDistanciaEventoFibra(zEmisores, elCOTDR.laFibra.longitudFibra/2, 1, x0Evento, y0Evento)
    tiemposVuelo = distancias/elCOTDR.laFibra.velocidadPropagacionPerturbaciones
    z, traza = elCOTDR.generaTraza()
    waterfall = np.array([traza])

    for tt in t:
        elCOTDR.laFibra.añadeDilatacionesLocales_evFijo(x0Evento, distancias, tiemposVuelo, p0=100000, coefAbsorcion=5, pasos = elCOTDR.laFibra.impulsoEscalon(t0Evento, tfEvento, 0.5, 0.25, 0.75, tt - tiemposVuelo))
        elCOTDR.laFibra.añadeRuidoTermico(intensidadRuidoTermico)
        z, traza = elCOTDR.generaTraza()
        waterfall = np.append(waterfall,[traza],axis=0)  # Matriz m, con trazas interferométricas en los distintos pasos de tiempo
"""
    waterfallVarianza = elCOTDR.generaWaterfallVarianza(waterfall = waterfall , intervaloVarianza = 20)
    #print(np.amax(waterfallVarianza), np.amin(waterfallVarianza))
    plt.imshow( 10 * waterfallVarianza, vmin=0, vmax=255, cmap='plasma')
    imagePath = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Varianza/Ruido + Evento Fijo/V" + str(n+1) + "_RuidoYEventoFijo.png"
    plt.imsave(imagePath, 10 * waterfallVarianza, vmin=0, vmax=255, cmap='plasma')
    filename = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Varianza/Ruido + Evento Fijo/V%d_RuidoYEventoFijo.npy" %(n+1)
    np.savetxt(filename, (10*waterfallVarianza).astype('float32'))
"""
    # Generación imagen Varianza

    fig=plt.figure(figsize=(2.5,2.5))
    plt.imshow(10*waterfallVarianza, vmin=0, vmax=255, cmap='plasma')
    plt.xticks(np.arange(0,501,100) , np.arange(0,101,20))
    plt.yticks(np.arange(0,502,83) , np.arange(0,61,10))
    plt.xlabel("$z_{fibra}$ (m)", fontsize=12)
    plt.ylabel("$t_{k}$ (s)", fontsize=12)
    plt.savefig('vevf.pdf')

In [ ]:
# 2.2) RUIDO + EVENTO FIJO | AJUSTE

elCOTDR = COTDR(longitudFibra=100, anchuraEmision=0, fMuestreo=1e9, fCorteDetector=1e8)
t = np.linspace(start=0.12, stop=60, num= int(60/0.12))
numeroImagenes = N

for n in np.arange(numeroImagenes):

    x0Evento = 10 + (elCOTDR.laFibra.longitudFibra-20) * np.random.random_sample()
    y0Evento = (elCOTDR.laFibra.longitudFibra/2 - 3) + 6 * np.random.random_sample()
    t0Evento = 0 + 15 * np.random.random_sample()
    tfEvento = 45 + 15 * np.random.random_sample()
    intensidadRuidoTermico = 4e-8 + 4e-8 * np.random.random_sample()
    print(x0Evento, y0Evento, t0Evento, tfEvento)
"""
    x0Evento = elCOTDR.laFibra.longitudFibra/2
    y0Evento = elCOTDR.laFibra.longitudFibra/2+1
    t0Evento = 0
    tfEvento = 60
    intensidadRuidoTermico = 6e-8
"""
    zEmisores = elCOTDR.laFibra.posEmisoresRayleigh
    distancias = elCOTDR.laFibra.calculaDistanciaEventoFibra(zEmisores, elCOTDR.laFibra.longitudFibra/2, 1, x0Evento, y0Evento)
    tiemposVuelo = distancias/elCOTDR.laFibra.velocidadPropagacionPerturbaciones
    z, traza = elCOTDR.generaTraza()
    waterfall = np.array([traza])

    for tt in t:
        elCOTDR.laFibra.añadeDilatacionesLocales_evFijo(x0Evento, distancias, tiemposVuelo, p0=100000, coefAbsorcion=5, pasos = elCOTDR.laFibra.impulsoEscalon(t0Evento, tfEvento, 0.5, 0.25, 0.75, tt - tiemposVuelo))
        elCOTDR.laFibra.añadeRuidoTermico(intensidadRuidoTermico)
        z, traza = elCOTDR.generaTraza()
        waterfall = np.append(waterfall,[traza],axis=0)  # Matriz m, con trazas interferométricas en los distintos pasos de tiempo
"""
    waterfallAjuste = elCOTDR.generaWaterfallAjuste(waterfall = waterfall , intervaloAjuste = 10, coeficienteAjuste = 1)
    #print(np.amax(waterfallAjuste), np.amin(waterfallAjuste))
    plt.imshow(5 * waterfallAjuste, vmin=0, vmax=255, cmap='plasma')
    imagePath = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Ajuste/Ruido + Evento Fijo/A" + str(n+1) + "_RuidoYEventoFijo.png"
    plt.imsave(imagePath, 5 * waterfallAjuste, vmin=0, vmax=255, cmap='plasma')
    filename = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Ajuste/Ruido + Evento Fijo/A%d_RuidoYEventoFijo.npy" %(n+1)
    np.savetxt(filename, (5*waterfallAjuste).astype('float32'))
"""
    # Generación imagen Ajuste

    fig=plt.figure(figsize=(2.5,2.5))
    plt.imshow(10*waterfallAjuste, vmin=0, vmax=255, cmap='plasma')
    plt.xticks(np.arange(0,501,100) , np.arange(0,101,20))
    plt.yticks(np.arange(0,492,81) , np.arange(0,61,10))
    plt.xlabel("$z_{fibra}$ (m)", fontsize=12)
    plt.ylabel("$t_{k}$ (s)", fontsize=12)
    plt.savefig('aevf.pdf')

In [ ]:
# 2.3) RUIDO + EVENTO FIJO | FOURIER

factorFourier = 2

elCOTDR = COTDR(longitudFibra=100, anchuraEmision=0, fMuestreo=1e9, fCorteDetector=1e8)
t = np.linspace(start=0.12, stop=60, num= int(60/0.12))
numeroImagenes = N

for n in np.arange(numeroImagenes):
    x0Evento = 10 + (elCOTDR.laFibra.longitudFibra-20) * np.random.random_sample()
    y0Evento = (elCOTDR.laFibra.longitudFibra/2 - 3) + 6 * np.random.random_sample()
    t0Evento = 0 + 15 * np.random.random_sample()
    tfEvento = 45 + 15 * np.random.random_sample()
    #intensidadRuidoTermico = 4e-8 + 4e-8 * np.random.random_sample()
    intensidadRuidoTermico = 4e-8
    #print(x0Evento, y0Evento, t0Evento, tfEvento)
"""
    x0Evento = elCOTDR.laFibra.longitudFibra/2
    y0Evento = elCOTDR.laFibra.longitudFibra/2+1
    t0Evento = 0
    tfEvento = 60
    intensidadRuidoTermico = 2e-8
"""
    zEmisores = elCOTDR.laFibra.posEmisoresRayleigh
    distancias = elCOTDR.laFibra.calculaDistanciaEventoFibra(zEmisores, elCOTDR.laFibra.longitudFibra/2, 1, x0Evento, y0Evento)
    tiemposVuelo = distancias/elCOTDR.laFibra.velocidadPropagacionPerturbaciones
    z, traza = elCOTDR.generaTraza()
    waterfall = np.array([traza])

    for tt in t:
        elCOTDR.laFibra.añadeDilatacionesLocales_evFijo(x0Evento, distancias, tiemposVuelo, p0=100000, coefAbsorcion=2, pasos = elCOTDR.laFibra.impulsoEscalon(t0Evento, tfEvento, 0.5, 0.25, 0.75, tt - tiemposVuelo))
        elCOTDR.laFibra.añadeRuidoTermico(intensidadRuidoTermico)
        z, traza = elCOTDR.generaTraza()
        waterfall = np.append(waterfall,[traza],axis=0)  # Matriz m, con trazas interferométricas en los distintos pasos de tiempo
"""
    waterfallFourier = elCOTDR.generaWaterfallFourier(waterfall = waterfall , intervaloFourier = 75)
    #print(np.amax(waterfallFourier), np.amin(waterfallFourier))
    #plt.imshow( factorFourier*waterfallFourier, vmin=0, vmax=255, cmap='plasma')
    imagePath = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Fourier/Ruido + Evento Fijo/F" + str(n+1) + "_RuidoYEventoFijo.png"
    plt.imsave(imagePath, factorFourier*waterfallFourier, vmin=0, vmax=255, cmap='plasma')
    filename = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Fourier/Ruido + Evento Fijo/F%d_RuidoYEventoFijo.npy" %(n+1)
    np.savetxt(filename, factorFourier*(waterfallFourier).astype('float32'))
"""
    # Generación imagen Fourier

    intervaloFourier=75
    fig=plt.figure(figsize=(2.5,2.5))
    plt.imshow(factorFourier*waterfallFourier, vmin=0, vmax=255, cmap='plasma')
    plt.xticks(np.arange(0,501,100) , np.arange(0,101,20))
    plt.yticks(np.arange(0,427,71) , np.arange(0,61,10))
    plt.xlabel("$z_{fibra}$ (m)", fontsize=12)
    plt.ylabel("$t_k$ (s)", fontsize=12)

### **2C - EVENTO MÓVIL ANDAR + RUIDO TÉRMICO**

In [ ]:
# 3.1) RUIDO + ANDAR PARALELO | VARIANZA

elCOTDR = COTDR(longitudFibra=100, anchuraEmision=0, fMuestreo=1e9, fCorteDetector=1e8)
t = np.linspace(start=0.12, stop=60, num= int(60/0.12))
numeroImagenes = N

tEntrePasos = 0.75
tSubida = 0.2
tPisada = 0.5


for n in np.arange(numeroImagenes):
    
    x0Evento = 10 + (elCOTDR.laFibra.longitudFibra-20) * np.random.random_sample()
    y0Evento = (elCOTDR.laFibra.longitudFibra/2 - 3) + 6 * np.random.random_sample()
    if x0Evento < 0.5*elCOTDR.laFibra.longitudFibra:
        vxEvento = 1
    else:
        vxEvento = -1
    vyEvento = 0
    t0Evento = 0 + 15 * np.random.random_sample()
    tfEvento = 45 + 15 * np.random.random_sample()
    intensidadRuidoTermico = 4e-8 + 4e-8 * np.random.random_sample()
    #print(x0Evento, y0Evento, t0Evento, tfEvento)
"""
    x0Evento = 10
    y0Evento = elCOTDR.laFibra.longitudFibra/2 + 1
    vxEvento = 1
    vyEvento = 0
    t0Evento = 0
    tfEvento = 60
    intensidadRuidoTermico = 4e-8
"""
    zEmisores = elCOTDR.laFibra.posEmisoresRayleigh
    distancias = elCOTDR.laFibra.calculaDistanciaEventoFibra(zEmisores, elCOTDR.laFibra.longitudFibra/2, 1, x0Evento, y0Evento)
    tiemposVuelo = distancias/elCOTDR.laFibra.velocidadPropagacionPerturbaciones
    z, traza = elCOTDR.generaTraza()
    waterfall = np.array([traza])

    for tt in t:
        xEvento = x0Evento + vxEvento*tt
        yEvento = y0Evento + vyEvento*tt
        distancias = elCOTDR.laFibra.calculaDistanciaEventoFibra(zEmisores, elCOTDR.laFibra.longitudFibra/2, 1, xEvento, yEvento)
        tiemposVuelo = distancias/elCOTDR.laFibra.velocidadPropagacionPerturbaciones
        elCOTDR.laFibra.añadeDilatacionesLocales_evFijo(xEvento, distancias, tiemposVuelo, p0=100000, coefAbsorcion=5, pasos = elCOTDR.laFibra.impulsoEscalon(t0Evento, tfEvento, tEntrePasos, tSubida, tPisada, tt - tiemposVuelo))
        elCOTDR.laFibra.añadeRuidoTermico(intensidadRuidoTermico)
        z, traza = elCOTDR.generaTraza()
        waterfall = np.append(waterfall,[traza],axis=0)  # Matriz m, con trazas interferométricas en los distintos pasos de tiempo
"""
    waterfallVarianza = elCOTDR.generaWaterfallVarianza(waterfall = waterfall , intervaloVarianza = 20)
    #print(np.amax(waterfallVarianza), np.amin(waterfallVarianza))
    #plt.imshow( 10 * waterfallVarianza, vmin=0, vmax=255, cmap='plasma')
    imagePath = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Varianza/Ruido + Andar en paralelo/V" + str(n+1) + "_RuidoYAndarParalelo.png"
    plt.imsave(imagePath, 10 * waterfallVarianza, vmin=0, vmax=255, cmap='plasma')
    filename = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Varianza/Ruido + Andar en paralelo/V%d_RuidoYAndarParalelo.npy" %(n+1)
    np.savetxt(filename, (10*waterfallVarianza).astype('float32'))
"""
    # Generación imagen Varianza

    fig=plt.figure(figsize=(2.5,2.5))
    plt.imshow(10*waterfallVarianza, vmin=0, vmax=255, cmap='plasma')
    plt.xticks(np.arange(0,501,100) , np.arange(0,101,20))
    plt.yticks(np.arange(0,502,83) , np.arange(0,61,10))
    plt.xlabel("$z_{fibra}$ (m)", fontsize=12)
    plt.ylabel("$t_k$ (s)", fontsize=12)

In [ ]:
# 3.2) RUIDO + ANDAR PARALELO | AJUSTE

elCOTDR = COTDR(longitudFibra=100, anchuraEmision=0, fMuestreo=1e9, fCorteDetector=1e8)
t = np.linspace(start=0.12, stop=60, num= int(60/0.12))
numeroImagenes = N

tEntrePasos = 0.75
tSubida = 0.2
tPisada = 0.5


for n in np.arange(numeroImagenes):
    
    x0Evento = 10 + (elCOTDR.laFibra.longitudFibra-20) * np.random.random_sample()
    y0Evento = (elCOTDR.laFibra.longitudFibra/2 - 3) + 6 * np.random.random_sample()
    if x0Evento < 0.5*elCOTDR.laFibra.longitudFibra:
        vxEvento = 1
    else:
        vxEvento = -1
    vyEvento = 0
    t0Evento = 0 + 15 * np.random.random_sample()
    tfEvento = 45 + 15 * np.random.random_sample()
    intensidadRuidoTermico = 4e-8 + 4e-8 * np.random.random_sample()
    #print(x0Evento, y0Evento, t0Evento, tfEvento)
"""
    x0Evento = 10
    y0Evento = elCOTDR.laFibra.longitudFibra/2 + 1
    vxEvento = 1
    vyEvento = 0
    t0Evento = 0
    tfEvento = 60
    intensidadRuidoTermico = 4e-8
"""
    zEmisores = elCOTDR.laFibra.posEmisoresRayleigh
    distancias = elCOTDR.laFibra.calculaDistanciaEventoFibra(zEmisores, elCOTDR.laFibra.longitudFibra/2, 1, x0Evento, y0Evento)
    tiemposVuelo = distancias/elCOTDR.laFibra.velocidadPropagacionPerturbaciones
    z, traza = elCOTDR.generaTraza()
    waterfall = np.array([traza])

    for tt in t:
        xEvento = x0Evento + vxEvento*tt
        yEvento = y0Evento + vyEvento*tt
        distancias = elCOTDR.laFibra.calculaDistanciaEventoFibra(zEmisores, elCOTDR.laFibra.longitudFibra/2, 1, xEvento, yEvento)
        tiemposVuelo = distancias/elCOTDR.laFibra.velocidadPropagacionPerturbaciones
        elCOTDR.laFibra.añadeDilatacionesLocales_evFijo(xEvento, distancias, tiemposVuelo, p0=100000, coefAbsorcion=5, pasos = elCOTDR.laFibra.impulsoEscalon(t0Evento, tfEvento, tEntrePasos, tSubida, tPisada, tt - tiemposVuelo))
        elCOTDR.laFibra.añadeRuidoTermico(intensidadRuidoTermico)
        z, traza = elCOTDR.generaTraza()
        waterfall = np.append(waterfall,[traza],axis=0)  # Matriz m, con trazas interferométricas en los distintos pasos de tiempo
"""
    waterfallAjuste = elCOTDR.generaWaterfallAjuste(waterfall = waterfall , intervaloAjuste = 10, coeficienteAjuste = 1)
    #print(np.amax(waterfallAjuste), np.amin(waterfallAjuste))
    #plt.imshow( 5* waterfallAjuste, vmin=0, vmax=255, cmap='plasma')
    imagePath = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Ajuste/Ruido + Andar en paralelo/A" + str(n+1) + "_RuidoYAndarParalelo.png"
    plt.imsave(imagePath, 5 * waterfallAjuste, vmin=0, vmax=255, cmap='plasma')
    filename = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Ajuste/Ruido + Andar en paralelo/A%d_RuidoYAndarParalelo.npy" %(n+1)
    np.savetxt(filename, (5*waterfallAjuste).astype('float32'))
"""
    # Generación imagen Ajuste

    fig=plt.figure(figsize=(2.5,2.5))
    plt.imshow(10*waterfallAjuste, vmin=0, vmax=255, cmap='plasma')
    plt.xticks(np.arange(0,501,100) , np.arange(0,101,20))
    plt.yticks(np.arange(0,492,81) , np.arange(0,61,10))
    plt.xlabel("$z_{fibra}$ (m)", fontsize=12)
    plt.ylabel("$t_k$ (s)", fontsize=12)
   

In [ ]:
# 3.3) RUIDO + ANDAR PARALELO | FOURIER

factorFourier = 1.5

elCOTDR = COTDR(longitudFibra=100, anchuraEmision=0, fMuestreo=1e9, fCorteDetector=1e8)
t = np.linspace(start=0.12, stop=60, num= int(60/0.12))
numeroImagenes = N

tEntrePasos = 0.75
tSubida = 0.2
tPisada = 0.5


for n in np.arange(numeroImagenes):
    
    x0Evento = 10 + (elCOTDR.laFibra.longitudFibra-20) * np.random.random_sample()
    y0Evento = (elCOTDR.laFibra.longitudFibra/2 - 3) + 6 * np.random.random_sample()
    if x0Evento < 0.5*elCOTDR.laFibra.longitudFibra:
        vxEvento = 1
    else:
        vxEvento = -1
    vyEvento = 0
    t0Evento = 0 + 15 * np.random.random_sample()
    tfEvento = 45 + 15 * np.random.random_sample()
    #intensidadRuidoTermico = 4e-8 + 4e-8 * np.random.random_sample()
    intensidadRuidoTermico = 4e-8
    #print(x0Evento, y0Evento, t0Evento, tfEvento)
"""
    x0Evento = 10
    y0Evento = elCOTDR.laFibra.longitudFibra/2 + 1
    vxEvento = 1
    vyEvento = 0
    t0Evento = 0
    tfEvento = 60
    intensidadRuidoTermico = 1e-8
"""
    zEmisores = elCOTDR.laFibra.posEmisoresRayleigh
    distancias = elCOTDR.laFibra.calculaDistanciaEventoFibra(zEmisores, elCOTDR.laFibra.longitudFibra/2, 1, x0Evento, y0Evento)
    tiemposVuelo = distancias/elCOTDR.laFibra.velocidadPropagacionPerturbaciones
    z, traza = elCOTDR.generaTraza()
    waterfall = np.array([traza])

    for tt in t:
        xEvento = x0Evento + vxEvento*tt
        yEvento = y0Evento + vyEvento*tt
        distancias = elCOTDR.laFibra.calculaDistanciaEventoFibra(zEmisores, elCOTDR.laFibra.longitudFibra/2, 1, xEvento, yEvento)
        tiemposVuelo = distancias/elCOTDR.laFibra.velocidadPropagacionPerturbaciones
        elCOTDR.laFibra.añadeDilatacionesLocales_evFijo(xEvento, distancias, tiemposVuelo, p0=100000, coefAbsorcion=2, pasos = elCOTDR.laFibra.impulsoEscalon(t0Evento, tfEvento, tEntrePasos, tSubida, tPisada, tt - tiemposVuelo))
        elCOTDR.laFibra.añadeRuidoTermico(intensidadRuidoTermico)
        z, traza = elCOTDR.generaTraza()
        waterfall = np.append(waterfall,[traza],axis=0)  # Matriz m, con trazas interferométricas en los distintos pasos de tiempo
"""
    waterfallFourier = elCOTDR.generaWaterfallFourier(waterfall = waterfall , intervaloFourier = 75)
    #print(np.amax(waterfallFourier), np.amin(waterfallFourier))
    #plt.imshow( factorFourier*waterfallFourier, vmin=0, vmax=255, cmap='plasma')
    imagePath = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Fourier/Ruido + Andar en paralelo/F" + str(n+1) + "_RuidoYAndarParalelo.png"
    plt.imsave(imagePath, factorFourier*waterfallFourier, vmin=0, vmax=255, cmap='plasma')
    filename = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Fourier/Ruido + Andar en paralelo/F%d_RuidoYAndarParalelo.npy" %(n+1)
    np.savetxt(filename, factorFourier*(waterfallFourier).astype('float32'))
"""
    # Generación imagen Fourier

    intervaloFourier=75
    fig=plt.figure(figsize=(2.5,2.5))
    plt.imshow(factorFourier*waterfallFourier, vmin=0, vmax=255, cmap='plasma')
    plt.xticks(np.arange(0,501,100) , np.arange(0,101,20))
    plt.yticks(np.arange(0,427,71) , np.arange(0,61,10))
    plt.xlabel("$z_{fibra}$ (m)", fontsize=12)
    plt.ylabel("$t_k$ (s)", fontsize=12)

### **2D - EVENTO MÓVIL CORRER + RUIDO TÉRMICO**

In [ ]:
# 4.1) RUIDO + CORRER PARALELO | VARIANZA

elCOTDR = COTDR(longitudFibra=100, anchuraEmision=0, fMuestreo=1e9, fCorteDetector=1e8)
t = np.linspace(start=0.12, stop=60, num= int(60/0.12))
numeroImagenes = N

tEntrePasos = 0.5
tSubida = 0.15
tPisada = 0.35


for n in np.arange(numeroImagenes):
    
    x0Evento = 10 + (elCOTDR.laFibra.longitudFibra-20) * np.random.random_sample()
    y0Evento = (elCOTDR.laFibra.longitudFibra/2 - 3) + 6 * np.random.random_sample()
    if x0Evento < 0.5*elCOTDR.laFibra.longitudFibra:
        vxEvento = 3
    else:
        vxEvento = -3
    vyEvento = 0
    t0Evento = 0 + 10 * np.random.random_sample()
    tfEvento = 50 + 10 * np.random.random_sample()
    intensidadRuidoTermico = 4e-8 + 4e-8 * np.random.random_sample()
    #print(x0Evento, y0Evento, t0Evento, tfEvento)
"""
    x0Evento = 10
    y0Evento = elCOTDR.laFibra.longitudFibra/2 + 1
    vxEvento = 3
    vyEvento = 0
    t0Evento = 0
    tfEvento = 60
    intensidadRuidoTermico = 4e-8
"""
    zEmisores = elCOTDR.laFibra.posEmisoresRayleigh
    distancias = elCOTDR.laFibra.calculaDistanciaEventoFibra(zEmisores, elCOTDR.laFibra.longitudFibra/2, 1, x0Evento, y0Evento)
    tiemposVuelo = distancias/elCOTDR.laFibra.velocidadPropagacionPerturbaciones
    z, traza = elCOTDR.generaTraza()
    waterfall = np.array([traza])

    for tt in t:
        xEvento = x0Evento + vxEvento*tt
        yEvento = y0Evento + vyEvento*tt
        distancias = elCOTDR.laFibra.calculaDistanciaEventoFibra(zEmisores, elCOTDR.laFibra.longitudFibra/2, 1, xEvento, yEvento)
        tiemposVuelo = distancias/elCOTDR.laFibra.velocidadPropagacionPerturbaciones
        elCOTDR.laFibra.añadeDilatacionesLocales_evFijo(xEvento, distancias, tiemposVuelo, p0=100000, coefAbsorcion=5, pasos = elCOTDR.laFibra.impulsoEscalon(t0Evento, tfEvento, tEntrePasos, tSubida, tPisada, tt - tiemposVuelo))
        elCOTDR.laFibra.añadeRuidoTermico(intensidadRuidoTermico)
        z, traza = elCOTDR.generaTraza()
        waterfall = np.append(waterfall,[traza],axis=0)  # Matriz m, con trazas interferométricas en los distintos pasos de tiempo
"""
    waterfallVarianza = elCOTDR.generaWaterfallVarianza(waterfall = waterfall , intervaloVarianza = 20)
    #print(np.amax(waterfallVarianza), np.amin(waterfallVarianza))
    #plt.imshow( 10 * waterfallVarianza, vmin=0, vmax=255, cmap='plasma')
    imagePath = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Varianza/Ruido + Correr en paralelo/V" + str(n+1) + "_RuidoYCorrerParalelo.png"
    plt.imsave(imagePath, 10 * waterfallVarianza, vmin=0, vmax=255, cmap='plasma')
    filename = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Varianza/Ruido + Correr en paralelo/V%d_RuidoYCorrerParalelo.npy" %(n+1)
    np.savetxt(filename, (10*waterfallVarianza).astype('float32'))
"""
    # Generación imagen Varianza

    fig=plt.figure(figsize=(2.5,2.5))
    plt.imshow(10*waterfallVarianza, vmin=0, vmax=255, cmap='plasma')
    plt.xticks(np.arange(0,501,100) , np.arange(0,101,20))
    plt.yticks(np.arange(0,502,83) , np.arange(0,61,10))
    plt.xlabel("$z_{fibra}$ (m)", fontsize=12)
    plt.ylabel("$t_k$ (s)", fontsize=12)

In [ ]:
# 4.2) RUIDO + CORRER PARALELO | AJUSTE

elCOTDR = COTDR(longitudFibra=100, anchuraEmision=0, fMuestreo=1e9, fCorteDetector=1e8)
t = np.linspace(start=0.12, stop=60, num= int(60/0.12))
numeroImagenes = N

tEntrePasos = 0.5
tSubida = 0.15
tPisada = 0.35


for n in np.arange(numeroImagenes):
    
    x0Evento = 10 + (elCOTDR.laFibra.longitudFibra-20) * np.random.random_sample()
    y0Evento = (elCOTDR.laFibra.longitudFibra/2 - 3) + 6 * np.random.random_sample()
    if x0Evento < 0.5*elCOTDR.laFibra.longitudFibra:
        vxEvento = 3
    else:
        vxEvento = -3
    vyEvento = 0
    t0Evento = 0 + 10 * np.random.random_sample()
    tfEvento = 50 + 10 * np.random.random_sample()
    intensidadRuidoTermico = 4e-8 + 4e-8 * np.random.random_sample()
    #print(x0Evento, y0Evento, t0Evento, tfEvento)
"""
    x0Evento = 10
    y0Evento = elCOTDR.laFibra.longitudFibra/2 + 1
    vxEvento = 3
    vyEvento = 0
    t0Evento = 0
    tfEvento = 60
    intensidadRuidoTermico = 4e-8
"""
    zEmisores = elCOTDR.laFibra.posEmisoresRayleigh
    distancias = elCOTDR.laFibra.calculaDistanciaEventoFibra(zEmisores, elCOTDR.laFibra.longitudFibra/2, 1, x0Evento, y0Evento)
    tiemposVuelo = distancias/elCOTDR.laFibra.velocidadPropagacionPerturbaciones
    z, traza = elCOTDR.generaTraza()
    waterfall = np.array([traza])

    for tt in t:
        xEvento = x0Evento + vxEvento*tt
        yEvento = y0Evento + vyEvento*tt
        distancias = elCOTDR.laFibra.calculaDistanciaEventoFibra(zEmisores, elCOTDR.laFibra.longitudFibra/2, 1, xEvento, yEvento)
        tiemposVuelo = distancias/elCOTDR.laFibra.velocidadPropagacionPerturbaciones
        elCOTDR.laFibra.añadeDilatacionesLocales_evFijo(xEvento, distancias, tiemposVuelo, p0=100000, coefAbsorcion=5, pasos = elCOTDR.laFibra.impulsoEscalon(t0Evento, tfEvento, tEntrePasos, tSubida, tPisada, tt - tiemposVuelo))
        elCOTDR.laFibra.añadeRuidoTermico(intensidadRuidoTermico)
        z, traza = elCOTDR.generaTraza()
        waterfall = np.append(waterfall,[traza],axis=0)  # Matriz m, con trazas interferométricas en los distintos pasos de tiempo
"""
    waterfallAjuste = elCOTDR.generaWaterfallAjuste(waterfall = waterfall , intervaloAjuste = 10, coeficienteAjuste = 1)
    #print(np.amax(waterfallAjuste), np.amin(waterfallAjuste))
    #plt.imshow( 5 * waterfallAjuste, vmin=0, vmax=255, cmap='plasma')
    imagePath = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Ajuste/Ruido + Correr en paralelo/A" + str(n+1) + "_RuidoYCorrerParalelo.png"
    plt.imsave(imagePath, 5 * waterfallAjuste, vmin=0, vmax=255, cmap='plasma')
    filename = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Ajuste/Ruido + Correr en paralelo/A%d_RuidoYCorrerParalelo.npy" %(n+1)
    np.savetxt(filename, (5*waterfallAjuste).astype('float32'))
"""
    # Generación imagen Ajuste

    fig=plt.figure(figsize=(2.5,2.5))
    plt.imshow(10*waterfallAjuste, vmin=0, vmax=255, cmap='plasma')
    plt.xticks(np.arange(0,501,100) , np.arange(0,101,20))
    plt.yticks(np.arange(0,492,81) , np.arange(0,61,10))
    plt.xlabel("$z_{fibra}$ (m)", fontsize=12)
    plt.ylabel("$t_k$ (s)", fontsize=12)
   

In [ ]:
# 4.3) RUIDO + CORRER PARALELO | FOURIER

factorFourier = 1.5

elCOTDR = COTDR(longitudFibra=100, anchuraEmision=0, fMuestreo=1e9, fCorteDetector=1e8)
t = np.linspace(start=0.12, stop=60, num= int(60/0.12))
numeroImagenes = N

tEntrePasos = 0.5
tSubida = 0.15
tPisada = 0.35


for n in np.arange(numeroImagenes):
    
    x0Evento = 10 + (elCOTDR.laFibra.longitudFibra-20) * np.random.random_sample()
    y0Evento = (elCOTDR.laFibra.longitudFibra/2 - 3) + 6 * np.random.random_sample()
    if x0Evento < 0.5*elCOTDR.laFibra.longitudFibra:
        vxEvento = 3
    else:
        vxEvento = -3
    vyEvento = 0
    t0Evento = 0 + 10 * np.random.random_sample()
    tfEvento = 50 + 10 * np.random.random_sample()
    #intensidadRuidoTermico = 4e-8 + 4e-8 * np.random.random_sample()
    intensidadRuidoTermico = 4e-8
    #print(x0Evento, y0Evento, t0Evento, tfEvento)
"""
    x0Evento = 10
    y0Evento = elCOTDR.laFibra.longitudFibra/2 + 1
    vxEvento = 3
    vyEvento = 0
    t0Evento = 0
    tfEvento = 60
    intensidadRuidoTermico = 2e-8
"""
    zEmisores = elCOTDR.laFibra.posEmisoresRayleigh
    distancias = elCOTDR.laFibra.calculaDistanciaEventoFibra(zEmisores, elCOTDR.laFibra.longitudFibra/2, 1, x0Evento, y0Evento)
    tiemposVuelo = distancias/elCOTDR.laFibra.velocidadPropagacionPerturbaciones
    z, traza = elCOTDR.generaTraza()
    waterfall = np.array([traza])

    for tt in t:
        xEvento = x0Evento + vxEvento*tt
        yEvento = y0Evento + vyEvento*tt
        distancias = elCOTDR.laFibra.calculaDistanciaEventoFibra(zEmisores, elCOTDR.laFibra.longitudFibra/2, 1, xEvento, yEvento)
        tiemposVuelo = distancias/elCOTDR.laFibra.velocidadPropagacionPerturbaciones
        elCOTDR.laFibra.añadeDilatacionesLocales_evFijo(xEvento, distancias, tiemposVuelo, p0=100000, coefAbsorcion=2, pasos = elCOTDR.laFibra.impulsoEscalon(t0Evento, tfEvento, tEntrePasos, tSubida, tPisada, tt - tiemposVuelo))
        elCOTDR.laFibra.añadeRuidoTermico(intensidadRuidoTermico)
        z, traza = elCOTDR.generaTraza()
        waterfall = np.append(waterfall,[traza],axis=0)  # Matriz m, con trazas interferométricas en los distintos pasos de tiempo
"""
    waterfallFourier = elCOTDR.generaWaterfallFourier(waterfall = waterfall , intervaloFourier = 75)
    #print(np.amax(waterfallFourier), np.amin(waterfallFourier))
    #plt.imshow(  factorFourier*waterfallFourier, vmin=0, vmax=255, cmap='plasma')
    imagePath = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Fourier/Ruido + Correr en paralelo/F" + str(n+1) + "_RuidoYCorrerParalelo.png"
    plt.imsave(imagePath, factorFourier*waterfallFourier, vmin=0, vmax=255, cmap='plasma')
    filename = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Fourier/Ruido + Correr en paralelo/F%d_RuidoYCorrerParalelo.npy" %(n+1)
    np.savetxt(filename, factorFourier*(waterfallFourier).astype('float32'))
"""
    # Generación imagen Fourier

    intervaloFourier=75
    fig=plt.figure(figsize=(2.5,2.5))
    plt.imshow(factorFourier*waterfallFourier, vmin=0, vmax=255, cmap='plasma')
    plt.xticks(np.arange(0,501,100) , np.arange(0,101,20))
    plt.yticks(np.arange(0,427,71) , np.arange(0,61,10))
    plt.xlabel("$z_{fibra}$ (m)", fontsize=12)
    plt.ylabel("$t_k$ (s)", fontsize=12)